<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Aufgabe4/Alina/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 4**

**1. Go to the following Git repo which contains a dataset of German articles:
https://github.com/tblock/10kGNAD**<br>
**2. Download the file train.csv from there and import it into a notebook.**

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import torch

In [4]:
import pandas as pd

#Read the CSV file with seperatr=; and set Columnnames 'labels' and 'text'
df = pd.read_csv('/content/drive/My Drive/KI_Lab/train.csv', error_bad_lines=False, sep=';', names=['labels','text'])

In [5]:
print(df)

             labels                                               text
0             Sport  21-Jähriger fällt wohl bis Saisonende aus. Wie...
1            Kultur  'Erfundene Bilder zu Filmen, die als verloren ...
2               Web  Der frischgekürte CEO Sundar Pichai setzt auf ...
3        Wirtschaft  Putin: "Einigung, dass wir Menge auf Niveau vo...
4            Inland  Estland sieht den künftigen österreichischen P...
...             ...                                                ...
9240         Inland  Bernd Saurer war Bridge-Juniorenweltmeister un...
9241  International  Sandhere soll in vergangener Woche bei Luftang...
9242     Wirtschaft  Derzeit Konzeptgruppe in Berlin – Kein Komment...
9243         Inland  Landeshauptmann will den vierten Regierungssit...
9244       Panorama  Er ist einer von Millionen syrischen Flüchtlin...

[9245 rows x 2 columns]


In [6]:
#From github
#url = 'https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv'
#df = pd.read_csv(url, error_bad_lines=False)

**3. Drop the label and only use the sentence in each line (the part after the first ; ).**

In [6]:
df = df.drop('labels', axis = 1)

,text
0,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,"'Erfundene Bilder zu Filmen, die als verloren ..."
2,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,Estland sieht den künftigen österreichischen P...
...,...
9240,Bernd Saurer war Bridge-Juniorenweltmeister un...
9241,Sandhere soll in vergangener Woche bei Luftang...
9242,Derzeit Konzeptgruppe in Berlin – Kein Komment...
9243,Landeshauptmann will den vierten Regierungssit...


**Install spacy on GPU**


In [7]:
!pip install -U spacy[cuda100] de


     |████████████████████████████████| 10.0MB 12.3MB/s 
     |████████████████████████████████| 2.1MB 44.2MB/s 
     |████████████████████████████████| 370.3MB 44kB/s 
  Created wheel for de: filename=de-0.1-cp36-none-any.whl size=1158 sha256=d447c43014439cef33643f1afe9a98885757ffddb115336d7608a6aa6fb2f489
  Stored in directory: /root/.cache/pip/wheels/fd/16/b9/46fac6c03128929538323f5457e947af752b7c58d4c7783f04
Successfully built de
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [8]:
!python -m spacy download de
import spacy 

     |████████████████████████████████| 14.9MB 17.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-cp36-none-any.whl size=14907580 sha256=82b3e25f7e0007ccf588a6404f2ee53958e742b3750a132bc386e7dd2295f91c
  Stored in directory: /tmp/pip-ephem-wheel-cache-pnp1pds4/wheels/db/f3/1e/0df0f27eee12bd1aaa94bcfef11b01eca62f90b9b9a0ce08fd
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [9]:
#Spacy on GPU
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

GPU: True


**4. Use spaCy to get all nouns in the whole file.**

In [10]:
nlp = spacy.load("de")

In [11]:
#Safe all nouns from 'text' in list_nouns
list_nouns =[]
df = df[0:200]
for index, row in df.iterrows():
  doc = nlp(row['text'])
  for token in doc:
    if token.pos_ == 'NOUN': 
      list_nouns.append(token.text)

In [12]:
print(list_nouns)

['21-Jähriger', 'Saisonende', 'Saisonende', 'Offensivspieler', 'Winter', 'Ried', '21-Jährige', '0:4-Heimdebakel', 'Samstag', 'Teilriss', 'Innenbandes', 'Knie', 'Magnetresonanz-Untersuchung', 'Donnerstag', 'Schiene', 'Pause', 'Bilder', 'Filmen', 'Ritt', 'Labyrinthe', 'Kinos', 'Filmgeschichte', 'Friedhof', 'Verlorenen', 'Begrabenen', 'Filme', 'Namen', 'Führungsteam', 'Atmosphäre', 'Mitarbeiter', 'Konkurrenz', 'Löhne', 'Mauscheleien', 'Untergebene', 'Ziele', 'Tür', 'Firmenebene', 'Umgangsformen', 'Wutausbrüche', 'Apple-Mitgründer', 'Geschäftspartner', 'Lebzeiten', 'Arschloch', 'Google-Mitgründer', 'Choleriker', 'Umgang', 'Mitarbeitern', 'Stelle', 'Zeiten', 'Management-Stils', 'Mitarbeiter', 'Geschäftspartner', 'Ex-Manager', 'Arschlöcher', 'Umstrukturierung', 'Gründung', 'Dachunternehmens', 'Suchmaschinisten', 'Monaten', 'Google-Chef', 'Umstrukturierungen', 'Lob', 'Branche', 'Android-Chef', 'Kollegen', 'Motorola-Präsident', 'Typen', 'Smartphone-Hersteller', 'Android', 'Zugang', 'Entwicklun

**5. Filter out the nouns that have characters wich are not in all_letters (see
code snippet below). The remaining set of nouns constitute our training data.**

In [1]:
import string
all_letters = string.ascii_letters + "ÄÖÜäöüß"
list_nouns_ascii = []

for noun in list_nouns:
  contained = True
  
  for char in noun:
    if char not in all_letters:
      contained = False
      break

  if contained:
    list_nouns_ascii.append(noun)

print(list_nouns_ascii)

NameError: ignored

**6. Go through the PyTorch tutorial on name generation. Clone the notebook, execute it
and understand what is going on.**<br>
**7. Use the same architecture to train a language model on the nouns that were extracted
from the German dataset in step 1 to 5. (Note: We do not need a „category“, you have
to remove everything that belongs to „category“ from the code).**

In [14]:
#Connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
#Define for Training and RNN needed variables
n_letters = len(all_letters) + 1 # Plus EOS marker

#Only need one categorie = all nouns from list_nouns
n_categories = 1

#dictionary for nouns
category_lines = {'NOUN': list_nouns_ascii}
all_categories = ['NOUN']

**RNN Class definiton**

In [24]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [25]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [26]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [27]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor.to(device), input_line_tensor.to(device), target_line_tensor.to(device)

In [28]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)

In [29]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

**Training of RNN**

In [30]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 1000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

**Sampling-Methods**

Original Sampling-Method

In [34]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter


        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('NOUN', 'HLCG')


Heeee
Lee
Cree
Geeee


**8. Extend the sampling logic in the sample function sucht that:
<br>
i. The model extends any given prefix string by first feeding the prefix into the RNN
and then start generating new characters. (Currently the model only samples based
on a given starting letter).**

In [36]:
# Sample from a category and starting letter
def sample_highest_score(category, start_letters):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letters)
        hidden = rnn.initHidden()

        output_name = start_letters

        #Loop over the Input Letters except the last one and create input tensors for each
        for character in start_letters[:-1]:
          input = inputTensor(character)
          output, hidden = rnn(category_tensor, input[0], hidden)

        #create Input Tensor for last character 
        input = inputTensor(start_letters[-1])

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            print(output.shape)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name


**ii. Currently the model uses the highest score during sampling. Change that into
drawing a real sample from the softmax distribution.**

In [78]:
#Convert all_letters to list and add <EOS> 
all_letters_norm = []
for character in all_letters:
  all_letters_norm.append(character)
all_letters_norm.append("<EOS>")

# Sample from a category and starting letter
def sample_real_sampling(category, start_letters):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letters)
        hidden = rnn.initHidden()

        output_name = start_letters

        #Loop over the Input Letters except the last one and create input tensors for each
        for character in start_letters[:-1]:
          input = inputTensor(character)
          output, hidden = rnn(category_tensor, input[0], hidden)

        #create Input Tensor for last character 
        input = inputTensor(start_letters[-1])

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)

            #Cast output Tensor to numpy array with dimension 1
            #Normalize output to get Values between 0 and 1 and get sum=1 for all values
            normalized_output = NormalizeData(output.cpu().detach().numpy().ravel())
            normalized_output = normalized_output / normalized_output.sum()
            print(normalized_output)
            
            top_char = random.choices(all_letters_norm, normalized_output, k=1)

            if top_char[0] == "<EOS>":
                break
            else:
                output_name += top_char[0]
            input = inputTensor(top_char)

        return output_name


In [39]:
sample_highest_score('NOUN', 'Ni')

'Nicher'

**9. Choose 10 random words from the training data and randomly cut-off a prefix. For
every of those prefixes give an output for:**
> **1. The model extending the prefix with the max score approach.**




In [32]:
# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

In [35]:
for i in range(10):
  word = randomChoice(list_nouns_ascii)
  print("Word: ", word)
  prefix = word[0:random.randint(1, len(word))]
  print("Prefix:", prefix)
  print(sample_highest_score('NOUN', word))



Word:  Jänner
Prefix: J
Jänner
Word:  Terroristen
Prefix: Terror
Terroristen
Word:  Auftritt
Prefix: Auftr
Auftritt
Word:  Mittwoch
Prefix: Mitt
Mittwoch
Word:  Satz
Prefix: Satz
Satze
Word:  Österreicher
Prefix: Österreic
Österreicher
Word:  Stockwerk
Prefix: Stock
Stockwerk
Word:  Lösung
Prefix: Lösung
Lösung
Word:  Dienstleistungen
Prefix: Dienstleis
Dienstleistungen
Word:  Monate
Prefix: Mon
Monate


In [79]:
for i in range(10):
  word = randomChoice(list_nouns_ascii)
  print("Word: ", word)
  prefix = word[0:random.randint(1, len(word))]
  print("Prefix:", prefix)

  string = "".join((random.choices(all_letters, k=2)))
  prefix = prefix + string
  print("Prefix + random Character: ", prefix)
  print(sample_real_sampling('NOUN', prefix))

Word:  Grenzwertgesetzgebung
Prefix: Grenzwertgesetzg
Prefix + random Character:  Grenzwertgesetzgdz
[0.03807456 0.01967662 0.02615696 0.02790202 0.04694528 0.02956948
 0.03850065 0.03402125 0.04028042 0.00302135 0.02204462 0.03469844
 0.02804919 0.04532513 0.03236438 0.01557474 0.00611998 0.04091646
 0.03845437 0.04307945 0.03615748 0.01413229 0.01608621 0.00871837
 0.00569634 0.02167405 0.00502831 0.00786512 0.00739511 0.00364508
 0.         0.01201681 0.0067277  0.00477884 0.00768624 0.01817524
 0.00425366 0.00478469 0.0050994  0.00639512 0.00446516 0.01817524
 0.00786652 0.00272402 0.00473031 0.01040429 0.01817524 0.00437666
 0.00784638 0.00294768 0.00756334 0.00524084 0.00233481 0.00341147
 0.0069974  0.01174545 0.00358417 0.01162675 0.00601699 0.05267579]
[0.03925485 0.01954229 0.0187606  0.02834243 0.04815469 0.03024876
 0.03984385 0.03467453 0.04132836 0.00276192 0.02279023 0.03654042
 0.02881992 0.04651018 0.03298827 0.01628746 0.00633955 0.0416325
 0.03860426 0.0442608  0.036

In [38]:
for i in range(10):
  word = randomChoice(list_nouns_ascii)
  print("Word: ", word)
  prefix = word[0:random.randint(1, len(word))]
  print("Prefix:", prefix)

  string = "".join((random.choices(all_letters, k=2)))
  prefix = prefix + string
  print("Prefix + random Character: ", prefix)
  print(sample_highest_score('NOUN', prefix))


Word:  Abschluss
Prefix: Ab
Prefix + random Character:  AbXr
torch.Size([1, 60])
torch.Size([1, 60])
AbXre
Word:  Überwachungskameras
Prefix: Überwachungsk
Prefix + random Character:  Überwachungsktu
torch.Size([1, 60])
Überwachungsktu
Word:  Bildschirm
Prefix: Bildschir
Prefix + random Character:  Bildschirjp
torch.Size([1, 60])
Bildschirjp
Word:  Kinder
Prefix: Kinde
Prefix + random Character:  KindeWI
torch.Size([1, 60])
KindeWI
Word:  Umfrage
Prefix: Umfrag
Prefix + random Character:  UmfragGx
torch.Size([1, 60])
UmfragGx
Word:  Hand
Prefix: Han
Prefix + random Character:  HanWj
torch.Size([1, 60])
HanWj
Word:  Hinteregger
Prefix: Hintereg
Prefix + random Character:  Hinteregqi
torch.Size([1, 60])
Hinteregqi
Word:  Botschafter
Prefix: Botsc
Prefix + random Character:  Botscjü
torch.Size([1, 60])
Botscjü
Word:  Studie
Prefix: Studie
Prefix + random Character:  StudieZL
torch.Size([1, 60])
StudieZL
Word:  Vorsitzender
Prefix: Vorsitzen
Prefix + random Character:  VorsitzenNb
torch.Si

In [39]:
import numpy as np

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [66]:
#Cast output Tensor to numpy array with dimension 1
#Normalize output to get Values between 0 and 1 and get sum=1 for all values
normalized_output = NormalizeData(output.cpu().detach().numpy().ravel())
normalized_output = normalized_output / normalized_output.sum()
print(normalized_output)

[0.03540353 0.01848854 0.01728065 0.02655502 0.04317021 0.02718289
 0.03700796 0.0317083  0.03832429 0.00285872 0.01728065 0.03324335
 0.02604899 0.04250061 0.03041673 0.01728065 0.00563806 0.03759925
 0.03587436 0.04107985 0.03354349 0.01300457 0.01565013 0.00884756
 0.00550856 0.02062719 0.00541394 0.01728065 0.01728065 0.00404643
 0.         0.01108433 0.00591641 0.00429179 0.01728065 0.00770556
 0.00422795 0.00459459 0.01728065 0.00719055 0.01728065 0.00445332
 0.00794153 0.01728065 0.0054947  0.01039536 0.00399127 0.00437606
 0.00742275 0.0031749  0.01728065 0.00548526 0.00221956 0.00325788
 0.00661717 0.01181302 0.01728065 0.01147957 0.00578621 0.0512505 ]


In [67]:
all_letters_norm = []
for character in all_letters:
  all_letters_norm.append(character)
all_letters_norm.append("<EOS>")

print(all_letters_norm)

random.choices(all_letters_norm, normalized_output, k=1)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Ä', 'Ö', 'Ü', 'ä', 'ö', 'ü', 'ß', '<EOS>']


['i']